In [1]:
#init
import torch, numpy, sys
sys.path.append("..")
from chatbot_end2end.module.interact_session import InteractSession
from nlptools.utils import Config
import torch.nn as nn

cfg = Config("../config/elsa_gru.yml")
session = InteractSession.build(cfg)

/usr/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.25.2) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


model config {'encoder': {'vocab_size': 30522, 'hidden_size': 768, 'intermediate_size': 1024, 'num_hidden_layers': 3, 'bidirectional': True}, 'decoder': {'num_hidden_layers': 3, 'intermediate_size': 1024, 'shared_embed': True, 'max_seq_len': 20, 'attention': True}}
load model from file data/cornell/model_gru.pt


In [2]:
skill_name = "cornell"
skill = session.topic_manager.skills[skill_name]
state_dict = skill.model.decoder.state_dict()
dialog_status = session.new_dialog()
dialog_status.add_utterance("I would hardly say you look like trash")
dialog_status.add_response("No thanks to you.")
data = dialog_status.status2data()

encoder_out, encoder_hidden = skill.model.dialog_embedding(data["utterance"].data, data["utterance_mask"].data, data["sentiment"].data)
utterance_mask = data["utterance_mask"].data
prev_output = data['response_cornell'].data[:, :-1]
target_output = data['response_cornell'].data[:, 1:]
target_output = target_output.unsqueeze(-1).contiguous().view(-1)
print(data["utterance"].data.shape)
print(encoder_out.shape, encoder_hidden.shape)
print(utterance_mask.shape)
print(prev_output.shape, target_output.shape)

torch.Size([1, 20])
torch.Size([1, 20, 768]) torch.Size([3, 1, 768])
torch.Size([1, 20])
torch.Size([1, 19]) torch.Size([19])


In [4]:
data

{'entity': PackedSequence(data=tensor([[nan, nan, nan,  ..., nan, nan, nan]]), batch_sizes=tensor([1]), sorted_indices=None, unsorted_indices=None),
 'utterance': PackedSequence(data=tensor([[  101,  1045,  2052,  6684,  2360,  2017,  2298,  2066, 11669,   102,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0]]), batch_sizes=tensor([1]), sorted_indices=None, unsorted_indices=None),
 'utterance_mask': PackedSequence(data=tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), batch_sizes=tensor([1]), sorted_indices=None, unsorted_indices=None),
 'reward': PackedSequence(data=tensor([[0.4750]]), batch_sizes=tensor([1]), sorted_indices=None, unsorted_indices=None),
 'sentiment': PackedSequence(data=tensor([[0.3612, 0.1779]]), batch_sizes=tensor([1]), sorted_indices=None, unsorted_indices=None),
 'response_cornell': PackedSequence(data=tensor([[ 101, 2053, 4283, 2000, 2017, 1012,  102,    0,    0,    0,    0,    0,
             0,    0,    0,  

In [5]:
skill.model.config

{'encoder': {'vocab_size': 30522,
  'hidden_size': 768,
  'intermediate_size': 1024,
  'num_hidden_layers': 3,
  'bidirectional': True},
 'decoder': {'num_hidden_layers': 3,
  'intermediate_size': 1024,
  'shared_embed': True,
  'max_seq_len': 20,
  'attention': True}}

In [9]:
import nlptools.zoo.encoders.gru
import chatbot_end2end.model.generative_tracker
import importlib
importlib.reload(nlptools.zoo.encoders.gru)
importlib.reload(chatbot_end2end.model.generative_tracker)

from nlptools.zoo.encoders.gru import GRUDecoder
from chatbot_end2end.model.generative_tracker import GenerativeTracker

embedding = skill.model.encoder.embedding

decoder_hidden_layers = skill.model.config["decoder"]["num_hidden_layers"]
max_seq_len = skill.model.config["decoder"]["max_seq_len"]
decoder_hidden_size = skill.model.config["decoder"]["intermediate_size"]

# bert_model_name = skill.model.config["bert_model_name"]
pad_id = skill.model.pad_id
bos_id = skill.model.bos_id
eos_id = skill.model.eos_id
unk_id = skill.model.unk_id
beam_size = 1
len_penalty = 1
unk_penalty = 1
dropout=0

shared_layers = {"encoder": skill.model.encoder}

tracker = GenerativeTracker(shared_layers=shared_layers, model_type="gru", skill_name=skill_name, decoder_hidden_laers=decoder_hidden_layers,
                             decider_hidden_size=decoder_hidden_size, max_seq_len=max_seq_len, dropout=dropout, pad_id=pad_id, bos_id=bos_id,
                            eos_id=eos_id, unk_id=unk_id, beam_size=beam_size, len_penalty=len_penalty, unk_penalty=unk_penalty)

decoder = GRUDecoder(embedding, num_hidden_layers=decoder_hidden_layers, intermediate_size=decoder_hidden_size, max_seq_len=max_seq_len, dropout=dropout)
decoder.load_state_dict(state_dict)
tracker.decoder = decoder

model config {'encoder': {'vocab_size': 30522, 'hidden_size': 768, 'intermediate_size': 1024, 'num_hidden_layers': 3, 'bidirectional': True}, 'decoder': {'num_hidden_layers': 1, 'intermediate_size': 1024, 'shared_embed': True, 'max_seq_len': 20, 'attention': True}}
tensor([[ 101, 3685, 2017, 2017, 2017, 2017, 2017, 2017, 2115, 9378, 2115, 2115,
         2115, 9378, 9378, 9378, 9378, 9378, 9378, 9378]]) [-0.62449539]
cannot you you you you you you your wash your your your wash wash wash wash wash wash


In [ ]:
result, score = tracker.beam_search(encoder_out, encoder_hidden, utterance_mask)
print(result, score)
response = result[0].detach().numpy()
response = response[response>0][1:-1]
response = skill.tokenizer.tokens2sentence(skill.vocab.id2words(response))
score = score.item()
print(response)

In [6]:
output = skill.model.decoder(prev_output, encoder_out=encoder_out, encoder_padding_mask=utterance_mask, encoder_hidden=encoder_hidden)
output_probs=skill.model.logsoftmax(output)
output_probs_expand = output_probs.contiguous().view(-1, output_probs.size(2))
loss = skill.model.loss_function(output_probs_expand, target_output)
print(output_probs_expand, target_output)
loss.item()

tensor([[-78.2895, -79.8654, -79.5300,  ..., -74.2396, -66.7165, -70.0399],
        [-55.1096, -55.3329, -55.7615,  ..., -48.5555, -43.9671, -42.6813],
        [-53.5853, -53.3650, -54.6049,  ..., -46.9380, -47.7513, -48.0000],
        ...,
        [-44.9050, -45.3993, -46.6519,  ..., -40.5495, -42.0737, -38.5874],
        [-44.8530, -45.3739, -46.6146,  ..., -40.4901, -42.0876, -38.5729],
        [-44.8117, -45.3576, -46.5866,  ..., -40.4416, -42.1141, -38.5585]],
       grad_fn=<ViewBackward>) tensor([2053, 4283, 2000, 2017, 1012,  102,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0])


11.852401733398438

In [3]:
skill.model

GenerativeTracker(
  (encoder): SentenceEncoder(
    (encoder): GRUEncoder(
      (embeddings): Embedding(30522, 768)
      (gru): GRU(768, 1024, num_layers=3, batch_first=True, bidirectional=True)
      (linear_out_x): Linear(in_features=2048, out_features=768, bias=True)
      (linear_out_hidden): Linear(in_features=2048, out_features=768, bias=True)
    )
    (embedding): Embedding(30522, 768)
  )
  (control_layer): Linear(in_features=770, out_features=768, bias=True)
  (decoder): GRUDecoder(
    (word_embedding): Embedding(30522, 768)
    (gru): GRU(768, 1024, num_layers=3, batch_first=True)
    (attn): Linear(in_features=1792, out_features=20, bias=True)
    (attn_combine): Linear(in_features=1536, out_features=768, bias=True)
    (dropout): Dropout(p=0)
    (hidden_proj): Linear(in_features=768, out_features=1024, bias=True)
    (intermediate_linear): Linear(in_features=1024, out_features=768, bias=True)
    (output_linear): Linear(in_features=768, out_features=30522, bias=False)